### Setup

1. Make a copy of `../.env.example` to `../.env` and fill the `OPENAI_API_KEY` and `CHAINLIT_API_KEY` environment variables.
2. This demo assumes that chainlit-cloud is running locally.
3. Run the cells to install the dependencies and the local sdk package.

In [1]:
pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -e ..

Obtaining file:///Users/thibautpatel/github/chainlit-sdks/python-client
  Preparing metadata (setup.py) ... done
  Attempting uninstall: chainlit-sdk
    Found existing installation: chainlit-sdk 0.1.0
    Uninstalling chainlit-sdk-0.1.0:
      Successfully uninstalled chainlit-sdk-0.1.0
  Running setup.py develop for chainlit-sdk

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Example

In [3]:
import asyncio
import json
import uuid

from chainlit_sdk import Chainlit
from chainlit_sdk.types import OperatorRole, StepType
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

sdk = Chainlit(batch_size=2)
sdk.instrument_openai()

ImportError: cannot import name 'Chainlit' from partially initialized module 'chainlit_sdk.chainlit_sdk' (most likely due to a circular import) (/Users/thibautpatel/github/chainlit-sdks/python-client/chainlit_sdk/chainlit_sdk.py)

In [ ]:
# Start a conversation

thread_id = uuid.uuid4()


@sdk.message(thread_id=thread_id)
def send_message(text):
    print(text)
    step = sdk.observer.get_current_step()
    step.output = text
    step.operatorRole = OperatorRole.ASSISTANT


@sdk.run(thread_id=thread_id)
def get_completion(input):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
            },
            {
                "role": "assistant",
                "content": welcome_message,
            },
            {
                "role": "user",
                "content": text,
            },
        ],
    )
    return completion


welcome_message = "What's your name? "
with sdk.step(type=StepType.MESSAGE, thread_id=thread_id) as step:
    step.output = welcome_message
    step.operatorRole = OperatorRole.SYSTEM

text = input(welcome_message)

with sdk.step(type=StepType.MESSAGE, thread_id=thread_id) as step:
    step.output = text
    step.operatorRole = OperatorRole.USER
    message = get_completion(text)
    send_message(message.choices[0].message.content)

In [ ]:
sdk.wait_until_queue_empty() # wait for all msgs to be in the cloud instance

In [ ]:
# Get the thread

print("\nSearching for the thread", thread_id, "...")
steps = await sdk.api.get_thread(id=str(thread_id))

thread = steps

print(json.dumps(thread, indent=2))